In [1]:
import pandas as pd
import joblib
import numpy as np

In [2]:
# Load the trained LightGBM model
lgb_model = joblib.load(r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\output\models\lightgbm_baseline_model.pkl")
threshold = joblib.load(r"C:\Users\Asus\Documents\GitHub\Credit-Scoring\output\models\best_threshold.pkl")


print("Mô hình và threshold đã được load thành công.")
print(f"Ngưỡng dự đoán hiện tại: {threshold:.3f}\n")

Mô hình và threshold đã được load thành công.
Ngưỡng dự đoán hiện tại: 0.143



In [3]:
FEATURE_CONFIG = [
    {
        "field": "age_years",
        "label": "Tuổi khách hàng",
        "description": "Tuổi thực tế của khách hàng. Tối thiểu 18, tối đa 75.",
        "unit": "năm",
        "range": (18, 75),
        "default": 35,
    },
    {
        "field": "employment_years",
        "label": "Số năm làm việc",
        "description": "Số năm khách hàng đã đi làm, kể cả thời gian làm việc không liên tục.",
        "unit": "năm",
        "range": (0, 50),
        "default": 5,
    },
    {
        "field": "credit_income_ratio",
        "label": "Hạn mức vay / thu nhập",
        "description": "Tổng hạn mức vay (bao gồm thấu chi, thẻ tín dụng) chia cho thu nhập hằng tháng.",
        "unit": "lần",
        "range": (0, 10),
        "default": 2.5,
    },
    {
        "field": "total_utilization",
        "label": "Dư nợ / hạn mức",
        "description": "Tỷ lệ đã sử dụng trên tổng hạn mức. >1 nghĩa là khách hàng đang vượt hạn mức.",
        "unit": "lần",
        "range": (0, 2),
        "default": 0.45,
    },
    {
        "field": "dpd_mean",
        "label": "Số ngày trễ hạn trung bình",
        "description": "Số ngày chậm trả trung bình trong 12 tháng gần nhất.",
        "unit": "ngày",
        "range": (0, 90),
        "default": 3,
    },
    {
        "field": "cc_avg_utilization",
        "label": "Tỷ lệ sử dụng thẻ tín dụng",
        "description": "Phần trăm tổng dư nợ thẻ tín dụng trên hạn mức cho phép.",
        "unit": "%",
        "range": (0, 100),
        "default": 42,
        "cast": lambda v: float(v) / 100 if float(v) > 1 else float(v),
        "postprocess": lambda v: v if v <= 1 else v / 100,
        "display_hint": "Nhập theo %, ví dụ 42 cho 42%.",
    },
]

In [4]:
def ask_value(cfg: dict) -> float:
    cast = cfg.get("cast", float)
    low, high = cfg.get("range", (None, None))
    default = cfg.get("default")
    unit = cfg.get("unit", "")
    extra = cfg.get("display_hint")
    prompt = f"{cfg['label']} ({cfg['field']}"
    if unit:
        prompt += f", đơn vị: {unit}"
    if low is not None and high is not None:
        prompt += f", gợi ý {low}–{high}"
    prompt += ")"
    if default is not None:
        prompt += f" [{default}]"
    prompt += ": "

    while True:
        raw = input(prompt).strip()
        if raw == "?":
            print(f"• {cfg['label']}: {cfg['description']}")
            if extra:
                print(f"  {extra}")
            continue
        if raw == "":
            if default is None:
                print("Vui lòng nhập số hoặc gõ '?' để xem giải thích.")
                continue
            value = default
        else:
            raw = raw.replace(",", ".")
            try:
                value = cast(raw)
            except ValueError:
                print("Giá trị không hợp lệ, vui lòng nhập số.")
                continue
        if low is not None and value < low:
            print(f"Giá trị nhỏ hơn mức tối thiểu ({low}).")
            continue
        if high is not None and value > high:
            print(f"Giá trị lớn hơn mức tối đa ({high}).")
            continue
        post = cfg.get("postprocess")
        return post(value) if post else value

print("Nhập thông tin khách hàng. Gõ '?' tại bất kỳ câu hỏi nào để xem mô tả.")
print("Giữ Enter để dùng giá trị gợi ý, giá trị được chuẩn hóa tự động.\n")

values = {cfg["field"]: ask_value(cfg) for cfg in FEATURE_CONFIG}
sample = pd.DataFrame([values])
sample = sample.reindex(columns=lgb_model.feature_name_, fill_value=0)

prob = float(lgb_model.predict_proba(sample)[:, 1])
pred = "KHÁCH HÀNG RỦI RO CAO" if prob >= threshold else "KHÁCH HÀNG AN TOÀN"

print("\n==============================")
print(f"🔢 Xác suất vỡ nợ: {prob:.2%}")
print(f"Kết luận: {pred}")
print("==============================\n")

Nhập thông tin khách hàng. Gõ '?' tại bất kỳ câu hỏi nào để xem mô tả.
Giữ Enter để dùng giá trị gợi ý, giá trị được chuẩn hóa tự động.

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50

🔢 Xác suất vỡ nợ: 94.05%
Kết luận: KHÁCH HÀNG RỦI RO CAO



C:\Users\Asus\AppData\Local\Temp\ipykernel_19208\978724690.py:52: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  prob = float(lgb_model.predict_proba(sample)[:, 1])
